In [ ]:
! pip install keras-tcn -q
! pip install --user tensorflow-addons -q
print('installed')

In [23]:
%tensorflow_version 2.10.1

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as keras_backend
from tensorflow.keras import layers, models, losses, metrics,optimizers, utils
import tensorflow_addons as tfa

import random
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

columns =  ['date','w_open_n', 'w_high_n', 'w_low_n', 'w_close_n', 'new_n', 'started_n','done_n', 'target_n', 'y_ystrdy_n']

file_hlp_path = './helpdesk-ohlc.csv'
df_hlp = pd.read_csv(file_hlp_path,parse_dates=['date'],index_col=['date'], usecols=columns)
df_hlp.sort_index(inplace=True)
df_hlp.dropna(inplace=True)

file_inc_path = './incident-ohlc.csv'
df_inc = pd.read_csv(file_inc_path,parse_dates=['date'],index_col=['date'], usecols=columns)
df_inc.sort_index(inplace=True)
df_inc.dropna(inplace=True)

random.seed(67)
np.random.seed(67)
tf.random.set_seed(67)
tf.keras.backend.set_epsilon(1)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [6]:
def convert_to_lookback(df, lookback_wind):
  y = df.dropna().target_n
  y_bsln = df.dropna().y_ystrdy_n

  X = df.dropna().drop(["target_n",'y_ystrdy_n'], axis=1)

  X= X.to_numpy()
  y= y.to_numpy()
  y_bsln = y_bsln.to_numpy()

  X_lb = []
  y_lb = []
  y_bsln_lb = []

  for i in tqdm(range(lookback_wind, len(X))):
    X_lb.append(X[i - lookback_wind:i])
    y_lb.append(y[i])
    y_bsln_lb.append(y_bsln[i])

  X_lb = np.array(X_lb)
  y_lb = np.array(y_lb)
  y_bsln_lb = np.array(y_bsln_lb)
  return X_lb, y_lb, y_bsln_lb

In [7]:
# datareader.py
class Window:
  def __init__(self, X,y,y_bsln):
    self.X = X
    self.y = y
    self.y_bsln = y_bsln

class MetaWindow:
  def __init__(self, support,query):
    # list of Windows 
    self.support = support
    self.query = query

  def add_support(self, win):
    self.support.append(win)

  def add_query(self, win):
    self.query.append(win)
  
class MetaTaskData:
  def __init__(self, metawins):
    # include ONE metawin for EACH meta task. if there are n_way = 10, this list includes 10 metawins
    self.tasks_metawin = metawins
  def n_tasks(self):
    return len(self.tasks_metawin)

class MAMLDataLoader:
    def __init__(self, X,y,y_bsln, meta_batch_size, n_way=10, k_shot = 1,q_query=1, n_step=1,n_rows=30):
        self.n_way = n_way
        self.k_shot = k_shot
        self.q_query = q_query
        self.n_rows = n_rows 

        self.X = X
        self.y = y
        self.y_bsln = y_bsln
        self.metawins = self.gen_metawins(k_shot,q_query,n_step,n_rows)

        self.meta_batch_size = meta_batch_size
        self.steps = len(self.metawins) // meta_batch_size

    def __len__(self):
        return len(self.metawins)

    def get_one_metatask_data(self):
        """
        Get a task, there are n_way classes in a task, each class has k_shot sheets for inner training, q_query sheets for outer training
        :return: support_data, query_data
        """
        metawins_nway = random.sample(self.metawins, self.n_way)
        return MetaTaskData(metawins_nway)

    def get_one_batch(self):
        """
        Get a batch of samples, where a batch is based on tasks as individuals
        :return: k_shot_data, q_query_data
        k_shot_data: k-shot means the number of images for each class
        q_query_data: q-query means the number of images in test (meta-test)

        """
        while True:
            batch_tasks_data = []
            for _ in range(self.meta_batch_size):
                batch_tasks_data.append(self.get_one_metatask_data())

            yield batch_tasks_data

    def gen_metawins(self,k_shot,q_query,n_step,n_rows):
      win_list = []
      for idx in range(0,len(self.X), n_step):
        win_list.append(Window(X=self.X[idx:idx+n_rows], y=self.y[idx:idx+n_rows], y_bsln=self.y_bsln[idx:idx+n_rows]))

      metawin_list = []
      n_metawins = len(win_list) - k_shot - q_query

      for idx in range(n_metawins):
        s = [win_list[i] for i in range(idx, idx+k_shot)]
        q = [win_list[j] for j in range(idx+k_shot, idx+k_shot+q_query)]
        metawin_list.append(MetaWindow(support= s ,query=q))

      random.shuffle(metawin_list)
      return metawin_list

    def __str__(self):
      return f" len(metwins):{len(self.metawins)},n_way:{self.n_way}, k_shot:{self.k_shot},q_query:{self.q_query}, n_rows:{self.n_rows }, len(X):{len(self.X)}, meta_batch_size:{self.meta_batch_size}"


In [8]:
# net.py
class Mish(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
      super().__init__(**kwargs)

  def call(self, inputs):
      return inputs * tf.math.tanh(tf.math.softplus(inputs))

  def get_config(self):
      base_config = super().get_config()
      return {**base_config}

  def compute_output_shape(self, input_shape):
      return input_shape



class MAML:
    def __init__(self, input_shape, args):
        self.input_shape = input_shape
        self.args = args 
        self.meta_model = self.get_maml_model()
    

    def get_maml_model(self):
        from tcn import TCN
        model = tf.keras.models.Sequential([
        TCN(input_shape=(self.args.lookback_wind, self.args.n_feature), nb_filters=self.args.nb_filters, dilations=self.args.dilations,activation=Mish()),
        tf.keras.layers.Dense(1, activation=Mish())
        ])
        return model 


    def train_on_batch(self,train_data, inner_optimizer, inner_step, outer_optimizer = None ):
      batch_acc = []
      batch_loss = []
      task_weights = []
      sup_loss = []
      sup_acc = []
      qu_loss = []
      qu_acc = [] 


      # Save the initial weight with meta_weights and set it as the weight of the inner step model
      meta_weights = self.meta_model.get_weights()

      metatask_data = next(train_data)[0]
      meta_support_X = [metatask_data.tasks_metawin[idx].support[idy].X  for idx in range(metatask_data.n_tasks()) for idy in range(len(metatask_data.tasks_metawin[idx].support))]
      meta_support_y = [metatask_data.tasks_metawin[idx].support[idy].y  for idx in range(metatask_data.n_tasks()) for idy in range(len(metatask_data.tasks_metawin[idx].support))]
      meta_query_X = [metatask_data.tasks_metawin[idx].query[idy].X  for idx in range(metatask_data.n_tasks()) for idy in range(len(metatask_data.tasks_metawin[idx].query))]
      meta_query_y = [metatask_data.tasks_metawin[idx].query[idy].y  for idx in range(metatask_data.n_tasks()) for idy in range(len(metatask_data.tasks_metawin[idx].query))]

      for support_X, support_y in zip(meta_support_X, meta_support_y):
        # Each task needs to load the most original weights to update
        self.meta_model.set_weights(meta_weights)
        for _ in range(inner_step):
            with tf.GradientTape() as tape:
              logits = self.meta_model(support_X, training=True)
              loss = losses.mean_squared_error(support_y, logits)
              loss = tf.reduce_mean(loss)
              acc = metrics.mean_absolute_percentage_error (support_y,tf.transpose(logits)) 
              acc = tf.reduce_mean(acc)
              sup_loss.append(loss)
              sup_acc.append(acc)

            grads = tape.gradient(loss, self.meta_model.trainable_variables)
            inner_optimizer.apply_gradients(zip(grads, self.meta_model.trainable_variables))

        # Every time the weights updated by the inner loop need to be saved once, to ensure that the same task is trained by the outer loop after the weights
        task_weights.append(self.meta_model.get_weights())

      with tf.GradientTape() as tape:
        for i, (query_X, query_y) in enumerate(zip(meta_query_X, meta_query_y)):
          # Load each task weights for forward propagation
          self.meta_model.set_weights(task_weights[i])
          logits = self.meta_model(query_X, training=True)
          loss = losses.mean_squared_error(query_y, logits)
          loss = tf.reduce_mean(loss)
          batch_loss.append(loss)
          acc = metrics.mean_absolute_percentage_error (query_y,tf.transpose(logits) ) 
          acc = tf.reduce_mean(acc)
          batch_acc.append(acc)
          qu_loss.append(loss)
          qu_acc.append(acc)

        mean_acc = tf.reduce_mean(batch_acc)
        mean_loss = tf.reduce_mean(batch_loss)

      # Whether it is updated or not, it is necessary to load the initial weight for update to prevent the val stage from changing the original weight
      self.meta_model.set_weights(meta_weights)
      
      if outer_optimizer:
        grads = tape.gradient(mean_loss, self.meta_model.trainable_variables)
        outer_optimizer.apply_gradients(zip(grads, self.meta_model.trainable_variables))
      
      return mean_loss, mean_acc, sup_loss, sup_acc, qu_loss, qu_acc

def clone_maml_model(mo: MAML):
    new_model = MAML( input_shape = mo.input_shape, args = mo.args)
    return new_model


In [9]:
class Args: 
  inner_lr=0.04
  outer_lr=0.001
  inner_optimizer=tfa.optimizers.LAMB(inner_lr)
  outer_optimizer=tfa.optimizers.LAMB(outer_lr)

  nb_filters = 64
  dilations=(1, 2, 4, 8, 16,32)
  kernel_size =3

  n_way=1
  inner_step = 1
  # inner_step = n_way

  k_shot=5
  n_query=3
  meta_batch_size = 1
  n_rows = 7
  n_step = n_rows
  input_shape=(None,7)

  n_feature = len(df_hlp.columns)-2
  lookback_wind=5
  epochs=10

args = Args()



In [ ]:
import time
from copy import deepcopy

# k-shot means the number of windows in each support set 
# n-query means the number of windows in each query set 

k_shot_s=[1,2,3,4,5,6,7,8,9,10]
n_query_s=[1,2,3,4,5,6,7,8,9,10]



args.epochs=1

maml = MAML(args.input_shape, args=args)
maml_c = deepcopy(maml)

filename = time.strftime("%Y%m%d-%H%M%S")
fp = open('maml-'+str(filename) + '.txt', 'w')
fp.close()


for k_shot in k_shot_s:
  for n_query in n_query_s:
    # for train_data_steps in train_data_steps_s:
    #   for val_data_steps in val_data_steps_s:
        args.k_shot = k_shot
        args.n_query = n_query
        print(f'args.k_shot:{args.k_shot}, args.n_query:{args.n_query}')


        X_hlp_lb, y_hlp_lb,y_bsln_hlp_lb = convert_to_lookback(df_hlp,lookback_wind= args.lookback_wind)
        X_inc_lb, y_inc_lb,y_bsln_inc_lb = convert_to_lookback(df_inc,lookback_wind= args.lookback_wind)

        # print(f'X_hlp_lb.shape:{X_hlp_lb.shape},y_hlp_lb:{y_hlp_lb.shape}, y_y_bsln_hlp_lb:{y_bsln_hlp_lb.shape} ')
        # print(f'X_inc_lb.shape:{X_inc_lb.shape},y_inc_lb:{y_inc_lb.shape}, y_y_bsln_inc_lb:{y_bsln_inc_lb.shape} ')

        args.meta_train ={'X':X_hlp_lb,'y':y_hlp_lb,'y_bsln':y_bsln_hlp_lb}
        args.meta_test={'X':X_inc_lb,'y':y_inc_lb,'y_bsln':y_bsln_inc_lb}  


        train_data = MAMLDataLoader(args.meta_train['X'],args.meta_train['y'],args.meta_train['y_bsln'], meta_batch_size=args.meta_batch_size,n_way = args.n_way,n_step= args.n_step, k_shot=args.k_shot , n_rows=args.n_rows)
        val_data = MAMLDataLoader(args.meta_test['X'],args.meta_test['y'],args.meta_test['y_bsln'], meta_batch_size=args.meta_batch_size,n_way = args.n_way,n_step= args.n_step, k_shot=args.k_shot , n_rows=args.n_rows)
        # number of tasks for training 
        # number of tasks for validataion 
        train_data.steps = 10
        val_data.steps = 1

        maml = clone_maml_model(maml_c)

        print(f'train_data.steps:{train_data.steps}, val_data.steps:{val_data.steps}')

        sup_loss_li =[]
        sup_acc_li=[]
        qu_loss_li = []
        qu_acc_li = []

        train_meta_loss = []
        train_meta_acc = []
        val_meta_loss = []
        val_meta_acc = []

        for e in range(args.epochs):
            train_progbar = utils.Progbar(train_data.steps)
            val_progbar = utils.Progbar(val_data.steps)
            print('\nEpoch {}/{}'.format(e+1, args.epochs))

            # maml = clone_maml_model(maml_c)

            for i in range(train_data.steps):

                batch_train_loss, train_acc,_,_,_,_ = maml.train_on_batch(train_data.get_one_batch(), args.inner_optimizer, inner_step=args.inner_step, outer_optimizer=args.outer_optimizer)

                train_meta_loss.append(batch_train_loss)
                train_meta_acc.append(train_acc)
                train_progbar.update(i+1, [('loss', np.mean(train_meta_loss)),('accuracy', np.mean(train_meta_acc))])

            for i in range(val_data.steps):
                batch_val_loss, val_acc, sup_loss, sup_acc, qu_loss, qu_acc  = maml.train_on_batch(val_data.get_one_batch(), args.inner_optimizer, inner_step=args.inner_step)

                val_meta_loss.append(batch_val_loss)
                val_meta_acc.append(val_acc)
                val_progbar.update(i+1, [('val_loss', np.mean(val_meta_loss)),('val_accuracy', np.mean(val_meta_acc))])

                sup_loss_li.append(sup_loss)
                sup_acc_li.append(sup_acc)
                qu_loss_li.append(qu_loss)
                qu_acc_li.append(qu_acc)

        qu_acc_res = [y.numpy() for x in qu_acc_li for y in x]
        qu_loss_res = [y.numpy() for x in qu_loss_li for y in x]
        sup_acc_res = [y.numpy() for x in sup_acc_li for y in x]
        sup_loss_res = [y.numpy() for x in sup_loss_li for y in x]

        fp = open('maml-'+str(filename)+f'_{str(k_shot).zfill(2)}{str(n_query).zfill(2)}' + '.txt', 'w')
        fp.write(f'qu_acc_{k_shot}{n_query}={qu_acc_res} \nqu_loss_{k_shot}{n_query}={qu_loss_res}\nsup_acc_{k_shot}{n_query}={sup_acc_res},\nsup_loss_{k_shot}{n_query}={sup_loss_res}\n')
        fp.close()

            # maml.meta_model.save_weights("maml.h5")
# fp.close()

print('... DONE ....')